## PART 1

In [20]:
with open('input.txt') as f:
    input = list(map(lambda x: x.strip(), f.readlines()))

PATHS = []

for l in input:

    pos, delta = l.split(' @ ') 
    x, y, _ = list(map(lambda x: int(x), pos.split(', ')))
    delta_x, delta_y, _ = list(map(lambda x: int(x), delta.split(', ')))

    m = delta_y / delta_x
    c = y - m*x

    PATHS.append([m,c,x,y,delta_x,delta_y])

res = 0
test_min, test_max = 200000000000000, 400000000000000

for i in range(len(PATHS)):
    for j in range(i+1, len(PATHS)):
        
        # parallel lines
        if (PATHS[j][0] - PATHS[i][0]) == 0:
            continue

        x_intersection = (PATHS[i][1] - PATHS[j][1]) / (PATHS[j][0] - PATHS[i][0])
        y_intersection = PATHS[i][0] * x_intersection + PATHS[i][1]
        
        
        # A incompatible
        if ((x_intersection > PATHS[i][2] and PATHS[i][4] < 0) or
            (x_intersection < PATHS[i][2] and PATHS[i][4] > 0) or
            (y_intersection > PATHS[i][3] and PATHS[i][5] < 0) or
            (y_intersection < PATHS[i][3] and PATHS[i][5] > 0)):
            continue
        
        # B incompatible
        if ((x_intersection > PATHS[j][2] and PATHS[j][4] < 0) or
            (x_intersection < PATHS[j][2] and PATHS[j][4] > 0) or
            (y_intersection > PATHS[j][3] and PATHS[j][5] < 0) or
            (y_intersection < PATHS[j][3] and PATHS[j][5] > 0)):
            continue
        
        # cross outside test area
        if not (test_min <= x_intersection <= test_max and 
                test_min <= y_intersection <= test_max):
            continue

        res += 1

res                        

11995

## PART 2